# Lesson 2: Tool Calling

## Setup

In [15]:
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [16]:
import nest_asyncio
nest_asyncio.apply()

## 1. Define a Simple ```Query Tool```

Tool calling enables LLMs to interact with external environments through a dynamic interface.
Tool calling not only helps **choosing the appropriate tool** but also **infer necessary arguments** for execution.
In standard RAG, LLMs are mainly used for synthesis of information only. 
Tool calling adds a layer of query understanding on top of a RAG pipeline, enables users to ask complex queries and get back more precise results. 

In the normal rag, we create a query engine as follows - 

```two indexes --> two query engines -->  two query engine tools --> one router query engine --> query() --> response```

1. create ```two indexes``` - vector index and summary index 

```python 
    summary_index = SummaryIndex(nodes)
    vector_index = VectorStoreIndex(nodes, embed_model=embed_model)
```
2. From the query index create ```two query engines``` - one for vector search and one for summarization. 

```python 
    summary_query_engine = SimpleQAE(query_index=summary_index)
    vector_query_engine = SimpleQAE(query_index=vector_index)
```
3. Using the query engines Create two ```query engine tools``` - one fore vector search and one fore summarization. 

```python 
    summary_tool = QueryEngineTool.from_defaults(
        query_engine=summary_query_engine,
        description=(
            "Useful for summarization questions related to the document"
        ),
    )
    
    vector_tool = QueryEngineTool.from_defaults(
        query_engine=vector_query_engine,
        description=(
            "Useful for retrieving specific context from the document."
        ),
    )
```
4. Using the query engine tools, create one ```router query engine`` that can route the queries to appropriate query tools, and hence he query indexes.

```python
   router_query_engine = RouterQueryEngine(
        selector=LLMSingleSelector.from_defaults(),
        query_engine_tools=[
            summary_tool,
            vector_tool,
        ],
        verbose=True
    )
```
5. Use the router query engine to create the response to query - 

```python 
    chat_response = router_query_engine.query(query) 
``` 

## ```Function Tool calling``` simple addition and arithmatic function

But in function tool calling, we do not have a query engine with the query tools. Rather we have just function calls as follows using the method `llm.predict_and_call()`

```NOTE``` - The functions have type annotations and doc strings as they will be used as a prompt for the LLM. 

In [17]:
from llama_index.core.tools import FunctionTool

def add(x: int, y: int) -> int:
    """Adds two integers together."""
    return x + y

def mystery(x: int, y: int) -> int: 
    """Mystery function that operates on top of two numbers."""
    return (x + y) * (x + y)


add_tool = FunctionTool.from_defaults(fn=add)
mystery_tool = FunctionTool.from_defaults(fn=mystery)

In [18]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")
response = llm.predict_and_call(
    [add_tool, mystery_tool], 
    "Tell me the output of the mystery on 2 and 9", 
    verbose=True
)
print(str(response))

=== Calling Function ===
Calling function: mystery with args: {"x": 2, "y": 9}
=== Function Output ===
121
121


## 2. Define an Auto-Retrieval Tool - Function tool calling example 2 - metadata filtering

### Load Data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

Here following is the objective - ```metadata filters```
1. get the metadata for the document. and extract the page numbers. 
2. This is done by embedding a filter on the query engine as follows - 
   1. create vector index
   2. create a query engine out of the index 
   3. add metadata filter to the query engine. 
3. Using metadata filters data retrieval can be be enhanced as follows - 
   1. meta deta filters are being added as a tool into the query engine. 
   2. this enables more precise data retrieval. 
   3. the LLM can intelligently infer relevant metadata filters (i.e page numbers) and use them to retrieve data, based on the user query. 
   4. you can define different types of metadata filters like section IDs, headers or footers. 

In [19]:
from llama_index.core import SimpleDirectoryReader
# load documents
documents = SimpleDirectoryReader(input_files=["../../../data/metagpt.pdf"]).load_data()

In [20]:
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [21]:
print(nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: ..\..\..\data\metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-11-06
last_modified_date: 2024-11-06

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
complex ta

In [21]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)

In [22]:
response = query_engine.query(
    "What are some high-level results of MetaGPT?")
print(str(response))

MetaGPT outperforms previous approaches in both HumanEval and MBPP benchmarks. It collaborates with GPT-4 to significantly improve performance in the HumanEval benchmark. Additionally, MetaGPT simplifies the process of transforming abstract requirements into detailed class and function designs through a specialized division of labor and SOPs workflow, leading to improved execution of code.


In [23]:
from llama_index.core.vector_stores import MetadataFilters

query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts(
        [
            {"key": "page_label", "value": "5"}
        ]
    )
)

response = query_engine.query(
    "What are some high-level results of MetaGPT?", 
)

In [24]:
print(str(response))

Some high-level results of MetaGPT include a structured software development process heavily reliant on Standard Operating Procedures (SOPs), starting from user requirements analysis to system design, task distribution, coding, quality assurance, and ultimately delivering a meticulously crafted software solution. Additionally, MetaGPT emphasizes the importance of structured communication interfaces in LLM-based multi-agent frameworks.


In [33]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '5', 'file_name': 'metagpt.pdf', 'file_path': '..\\..\\..\\data\\metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-11-06', 'last_modified_date': '2024-11-06'}


### Define the Auto-Retrieval Tool as a function tool that can retrieve from a specific page or any page. 

In [25]:
from typing import List
from llama_index.core.vector_stores import FilterCondition


def vector_query(
    query: str, 
    page_numbers: List[str]
) -> str:
    """Perform a vector search over an index.
    
    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.
    
    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]
    
    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response
    

vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)

In [26]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
response = llm.predict_and_call(
    [vector_query_tool], 
    "What are the high-level results of MetaGPT as described on page 2, 3, and 4?", 
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "high-level results of MetaGPT", "page_numbers": ["2", "3", "4"]}
=== Function Output ===
MetaGPT achieves state-of-the-art performance on HumanEval and MBPP, showcasing its effectiveness as a meta-programming framework for developing LLM-based multi-agent systems.


In [37]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '2', 'file_name': 'metagpt.pdf', 'file_path': '..\\..\\..\\data\\metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-11-06', 'last_modified_date': '2024-11-06'}
{'page_label': '4', 'file_name': 'metagpt.pdf', 'file_path': '..\\..\\..\\data\\metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-11-06', 'last_modified_date': '2024-11-06'}


## Let's add some other tools! - summary + vector query

In [34]:
import geopandas as gpd
import folium
def render_graph():

    """Renders and visualizes the flight plan."""
    
    # Read the KML file
    gdf = gpd.read_file('../../../data/UIAA-UNNT.kml')

    # Calculate the centroid of the geometries
    centroid = gdf.geometry.centroid

    # Create a Folium map
    m = folium.Map(location=[centroid.y.mean(), centroid.x.mean()], zoom_start=10)


    # Add the flight path to the map
    folium.GeoJson(gdf, style_function=lambda x: {'color': 'blue', 'weight': 3}).add_to(m)

    # Display the map
    m

In [36]:
graph_tool = FunctionTool.from_defaults(
    name="graph_tool",
    fn=render_graph)

In [37]:
render_graph()

C:\Users\samra\AppData\Local\Temp\ipykernel_64220\3795613200.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gdf.geometry.centroid


In [28]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of MetaGPT"
    ),
)

In [29]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool, graph_tool], 
    "What are the MetaGPT comparisons with ChatDev described on page 8?", 
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "MetaGPT comparisons with ChatDev", "page_numbers": ["8"]}
=== Function Output ===
MetaGPT outperforms ChatDev on the challenging SoftwareDev dataset in nearly all metrics. For example, MetaGPT achieves a higher score in executability, takes less time for software generation, uses more tokens but requires fewer tokens to generate one line of code compared to ChatDev. Additionally, MetaGPT demonstrates autonomous software generation capabilities and highlights the benefits of using SOPs in collaborations between multiple agents.


In [ ]:
for n in response.source_nodes:
    print(n.metadata)

In [ ]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "What is a summary of the paper?", 
    verbose=True
)

In [39]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool, graph_tool], 
    "render the flight plan", 
    verbose=True
)

=== Calling Function ===
Calling function: graph_tool with args: {}
=== Function Output ===
None


C:\Users\samra\AppData\Local\Temp\ipykernel_64220\3795613200.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gdf.geometry.centroid
